<a href="https://colab.research.google.com/github/Vinicius-Jose/Notebooks/blob/main/ChattbotAndVoicebot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Instalando Dependencias

In [ ]:
!apt install libasound2-dev portaudio19-dev libportaudio2 libportaudiocpp0 ffmpeg
!pip install SpeechRecognition
!pip install pyaudio
!pip install ffmpeg-python
!pip install gTTS
!pip install chatterbot
!pip install chatterbot-corpus

Reading package lists... Done
Building dependency tree       
Reading state information... Done
libportaudio2 is already the newest version (19.6.0-1).
libportaudiocpp0 is already the newest version (19.6.0-1).
portaudio19-dev is already the newest version (19.6.0-1).
libasound2-dev is already the newest version (1.1.3-5ubuntu0.5).
ffmpeg is already the newest version (7:3.4.8-0ubuntu0.2).
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 34 not upgraded.


# Importaçoes necessarias

In [ ]:
import speech_recognition as sr
from io import BytesIO
from base64 import b64decode
from google.colab import output
from IPython.display import Javascript
from IPython.display import HTML, Audio
from google.colab.output import eval_js
import numpy as np
from scipy.io.wavfile import read as wav_read
import io
import ffmpeg
from time import sleep
from scipy.io.wavfile import write

from gtts import gTTS, lang
from tempfile import TemporaryFile
from IPython.core.display import display

from chatterbot import ChatBot
from chatterbot.trainers import ChatterBotCorpusTrainer
from chatterbot import logic, comparisons

# TTS


In [ ]:
def tts(text,lang='pt',tld='com.br'):
  response = gTTS(text ,lang=lang,tld=tld)
  response.save("tts.wav")
  display(Audio("tts.wav", autoplay=True)) 
  sleep(6)

# Speech Recognition

In [ ]:
AUDIO_HTML = """
<script>
var my_div = document.createElement("DIV");
var my_p = document.createElement("P");
var my_btn = document.createElement("BUTTON");
var t = document.createTextNode("Press to start recording");

my_btn.appendChild(t);
//my_p.appendChild(my_btn);
my_div.appendChild(my_btn);
document.body.appendChild(my_div);

var base64data = 0;
var reader;
var recorder, gumStream;
var recordButton = my_btn;

var handleSuccess = function(stream) {
  gumStream = stream;
  var options = {
    //bitsPerSecond: 8000, //chrome seems to ignore, always 48k
    mimeType : 'audio/webm;codecs=opus'
    //mimeType : 'audio/webm;codecs=pcm'
  };            
  //recorder = new MediaRecorder(stream, options);
  recorder = new MediaRecorder(stream);
  recorder.ondataavailable = function(e) {            
    var url = URL.createObjectURL(e.data);
    var preview = document.createElement('audio');
    preview.controls = true;
    preview.src = url;
    document.body.appendChild(preview);

    reader = new FileReader();
    reader.readAsDataURL(e.data); 
    reader.onloadend = function() {
      base64data = reader.result;
      //console.log("Inside FileReader:" + base64data);
    }
  };
  recorder.start();
  };

recordButton.innerText = "Recording... press to stop";

navigator.mediaDevices.getUserMedia({audio: true}).then(handleSuccess);


function toggleRecording() {
  if (recorder && recorder.state == "recording") {
      recorder.stop();
      gumStream.getAudioTracks()[0].stop();
      recordButton.innerText = "Saving the recording... pls wait!"
  }
}

// https://stackoverflow.com/a/951057
function sleep(ms) {
  return new Promise(resolve => setTimeout(resolve, ms));
}

var data = new Promise(resolve=>{
//recordButton.addEventListener("click", toggleRecording);
recordButton.onclick = ()=>{
toggleRecording()

sleep(2000).then(() => {
  // wait 2000ms for the data to be available...
  // ideally this should use something like await...
  //console.log("Inside data:" + base64data)
  resolve(base64data.toString())

});

}
});
      
</script>
"""

In [ ]:
def get_audio():
  display(HTML(AUDIO_HTML))
  data = eval_js("data")
  binary = b64decode(data.split(',')[1])
  
  process = (ffmpeg
    .input('pipe:0')
    .output('pipe:1', format='wav')
    .run_async(pipe_stdin=True, pipe_stdout=True, pipe_stderr=True, quiet=True, overwrite_output=True)
  )
  output, err = process.communicate(input=binary)
  
  riff_chunk_size = len(output) - 8
  # Break up the chunk size into four bytes, held in b.
  q = riff_chunk_size
  b = []
  for i in range(4):
      q, r = divmod(q, 256)
      b.append(r)

  # Replace bytes 4:8 in proc.stdout with the actual size of the RIFF chunk.
  riff = output[:4] + bytes(b) + output[8:]

  sr2, audio = wav_read(io.BytesIO(riff))

  return audio, sr2

In [ ]:
def grava_audio(file_name):
  samplerate = 44100; fs = 100

  t = np.linspace(0., 1., samplerate)

  amplitude = np.iinfo(np.int16).max

  data = amplitude * np.sin(2. * np.pi * fs * t)
  data, sr2 = get_audio()

  write(file_name, samplerate, data.astype(np.int16)) 

In [ ]:
def reconhecimento():
  print('Lendo audio: \n')
  texto = None
  file_name = "reconhecimento.wav"
  grava_audio(file_name)
  r = sr.Recognizer()

  with sr.WavFile(file_name) as fonte:
      audio = r.record(fonte)

      texto = r.recognize_google(audio, language='pt-BR')

      try:
          print(texto)
          print('Áudio reconhecido GOOGLE: \n ' + texto.capitalize() + '\n')

      except Exception:
          print('Nao foi possivel compreender o audio')
  return texto



# Chatterbot


In [ ]:
bot = ChatBot('Tony',logic_adapters=[{
        "import_path": "chatterbot.logic.BestMatch",
        "statement_comparison_function": comparisons.LevenshteinDistance}
    ])

In [ ]:
trainer = ChatterBotCorpusTrainer(bot)
trainer.train("chatterbot.corpus.portuguese")

Training compliment.yml: [####################] 100%
Training conversations.yml: [####################] 100%
Training greetings.yml: [####################] 100%
Training linguistic_knowledge.yml: [####################] 100%
Training proverbs.yml: [####################] 100%
Training suggestions.yml: [####################] 100%
Training trivia.yml: [####################] 100%
Training unilab.yml: [####################] 100%


In [ ]:
trainer.export_for_training('./my_export.json')

# Execuçao do Chatbott e VoiceBot

In [ ]:
def chatbot():
  while True:
    pergunta = input("Usuário: ")
    resposta = bot.get_response(pergunta)
    if float(resposta.confidence) > 0.5:
        print('TW Bot: ', resposta)
    else:
        print('TW Bot: Ainda não sei responder esta pergunta')


In [ ]:
def voicebot():
  while True:
    pergunta = reconhecimento()
    resposta = bot.get_response(pergunta)
    if float(resposta.confidence) > 0.5:
        tts(resposta.text)
    else:
        tts('Ainda não sei responder esta pergunta')

In [ ]:
a = int(input('Voce deseja conversar por voz ou texto? Digite: 1-Voz 2-Texto '))

if a == 1:
  voicebot()
elif a == 2:
  chatbot()
else:
  print('Opcao invalida encerrando sistema')
